# Cube with Planar Varrying Cross Section

## Cross Section Choice

For this model, I have chose to use a total cross section defined as

$$\Sigma_t(\boldsymbol{r}) = x + y + z + 1.$$

This cross section will be valid over a $2cm \times 2cm \times 2cm$ cube, with a lower corner at $(0,0,0)$ and and upper corner at (2,2,2). It will be a single energy group, non-multiplying system, with the following scatter and capture cross sections:

$$ \frac{\Sigma_s(\boldsymbol{r})}{\Sigma_t(\boldsymbol{r})} = 0.7 \qquad \frac{\Sigma_\gamma(\boldsymbol{r})}{\Sigma_t(\boldsymbol{r})} = 0.3$$

The cube will be divided into 32 sections along each axis, creating 32768 smaller cubes, in which the cross sections shall be constant. As such, a separate cross section definition must be created for each smaller cube. The following code does just that for openMC, using the cross section values at the center of a given cube.

In [1]:
import os
import numpy as np
import openmc

%matplotlib inline

In [11]:
# Define the interval for the single group
E_groups = openmc.mgxs.EnergyGroups(np.logspace(-5,7,3))

# Loop to generate XS data for each cube
d = 2.0/32.0
xs_data_file = openmc.MGXSLibrary(E_groups)
for i in range(32):
    for j in range(32):
        for k in range(32):
            x = i*d + (d/2.0)
            y = j*d + (d/2.0)
            z = k*d + (d/2.0)
            XSname = str(i) + '.' + str(j) + '.' + str(k)
            XS = openmc.XSdata(XSname, E_groups)
            XS.order = 0
            Et = [(x + y + z + 1.0),0.0]
            Es = [[[0.7*Et[0], 0.0],
                   [0.0, 0.0]]]
            Es = np.array(Es)
            Es = np.rollaxis(Es, 0, 3)
            Ec = [0.3*Et[0], 0.0]
            XS.set_total(Et, temperature=294.)
            XS.set_absorption(Ec, temperature=294.)
            XS.set_scatter_matrix(Es, temperature=294.)
            xs_data_file.add_xsdata(XS)
            
# Write XS data to .h5 file
xs_data_file.export_to_hdf5('cube_plane_xs.h5')

# Now must define materials
materials = {}
for i in range(32):
    for j in range(32):
        for k in range(32):
            XSname = str(i) + '.' + str(j) + '.' + str(k)
            materials[XSname] = openmc.Material(name = XSname)
            materials[XSname].set_density('macro', 1.)
            materials[XSname].add_macroscopic(XSname)
            
materials_file = openmc.Materials(materials.values())
materials_file.cross_sections = 'cube_plane_xs,h5'
materials_file.export_to_xml()